# Subway Data Analysis

## Introduction

O sistema de ônibus e trens de Nova Iorque - o Metro Transit Authority - [fornece seus dados para download](http://web.mta.info/developers/developer-data-terms.html#data) através de  arquivos CSV. Dentre as informações disponíveis estão os **registros semanais de dados das catracas do metrô**. 


Estes registros contém contagens cumulativas das entradas e saídas, normalmente agrupadas em períodos de 4 horas, com dados adicionais que permitem identificar a estação e catraca específica correspondente a cada linha do arquivo. Neste projeto iremos utilizar um desses registros, mas não precisa baixar nada agora! O primeiro exercício será escrever um código Python para fazer isso por você :-)




# Sobre este projeto

Neste projeto você irá aplicar todos os conhecimentos adquiridos neste primeiro mês de curso, com tarefas básicas de aquisição e limpeza de dados. No processo iremos descobrir informações essenciais sobre os dados, utilizando o que foi aprendido no curso de estatística. 

O objetivo deste projeto é explorar a relação entre os dados das catracas do metrô de Nova Iorque e o clima no dia da coleta. Para isso, além dos dados do metrô, precisaremos dos dados de clima da cidade de Nova Iorque. 

Os principais pontos que serão verificados neste trabalho:

- Coleta de dados da internet
- Utilização de estatística para análise de dados
- Manipulação de dados e criação de gráficos simples com o `Pandas`

*Como conseguir ajuda*: Sugerimos que busque apoio nos canais abaixo, na seguinte ordem de prioridade:

| Tipo de dúvida\Canais         	| Google 	| Fórum 	| Slack 	| Email 	|
|-------------------------------	|--------	|-------	|-------	|-------	|
| Programação Python e Pandas    	| 1      	| 2     	| 3     	|       	|
| Requisitos do projeto         	|        	| 1     	| 2     	| 3     	|
| Partes específicas do Projeto 	|        	| 1     	| 2     	| 3     	|

Os endereços dos canais são:

- Fórum: https://discussions.udacity.com/c/ndfdsi-project
- Slack: [udacity-br.slack.com](https://udacity-br.slack.com/messages/C5MT6E3E1)
- Email: data-suporte@udacity.com

**Espera-se que o estudante entregue este relatório com:**

- Todos os exercícios feitos, com atenção especial para os trechos de código a completar (sinalizados com `# your code here`), pois eles são essenciais para que o código rode corretamente
- O arquivo ipynb exportado como HTML

Para entregar este projeto envie este `.ipynb` preenchido e o HTML, zipados, na página correspondente da sala de aula.

# Sobre o dataset

Descrição das colunas
<pre>
C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
  
C/A      = Agrupamento de catracas de que a catraca faz parte (_Control Area_)
UNIT     = Cabine de controle associada à estação onde a catraca se encontra (_Remote Unit for a station_)
SCP      = Endereço específico da catraca (_Subunit Channel Position_)
STATION  = Nome da estação onde a catraca se encontra
LINENAME = Código representando todas linhas que passam na estação*
DIVISION = Código representando a concessionária original da linha, antes da prefeitura assumir a gestão   
DATE     = Representa a data (no formato MM-DD-YY) do evento de auditoria agendado
TIME     = Representa o horário (hh:mm:ss) do evento de auditoria agendado
DESc     = Descreve o tipo de evento de auditoria registrado:
           1. "REGULAR" representando um evento de auditoria padrão, em que a contagem é feita a cada 4 horas
           2. "RECOVR AUD" significa que o valor específico estava perdido, mas foi recuperado posteriormente 
           3. Diversos códigos sinalizam situações em que auditorias são mais frequentes devido a atividades de
              planejamento ou solução de problemas. 
ENTRIES  = A contagem cumulativa de entradas associadas à catraca desde o último registro
EXITS    = A contagem cumulativa de saídas associadas à catraca desde o último registro

*  Normalmente as linhas são representadas por um caractere. LINENAME 456NQR significa que os trens 4, 5, 6, N, Q e R passam pela estação.
</pre>

# Lembretes

Antes de começarmos, alguns lembretes devem ter em mente ao usar os notebooks iPython:

- Lembre-se de que você pode ver do lado esquerdo de uma célula de código quando foi executado pela última vez se houver um número dentro das chaves.
- Quando você inicia uma nova sessão do notebook, certifique-se de executar todas as células até o ponto em que você deixou a última vez. Mesmo que a saída ainda seja visível a partir de quando você executou as células em sua sessão anterior, o kernel começa em um estado novo, então você precisará recarregar os dados, etc. em uma nova sessão.
- O ponto anterior é útil para ter em mente se suas respostas não correspondem ao que é esperado nos questionários da aula. Tente recarregar os dados e execute todas as etapas de processamento um a um para garantir que você esteja trabalhando com as mesmas variáveis e dados que estão em cada fase do questionário.

## Seção 1 - Coleta de Dados

### *Exercicio 1.1*

Mãos a obra!! Agora é sua vez de coletar os dados. Escreva abaixo um código python que acesse o link http://web.mta.info/developers/turnstile.html e baixe os arquivos do mês de junho de 2017. O arquivo deverá ser salvo com o nome turnstile_100617.txt onde 10/06/17 é a data do arquivo.

Abaixo seguem alguns comandos que poderão te ajudar:

Utilize a biblioteca **urllib** para abrir e resgatar uma página da web. Utilize o comando abaixo onde **url** será o caminho da página da web onde se encontra o arquivo:

```python
u = urllib.urlopen(url)
html = u.read()
```

Utilize a biblioteca **BeautifulSoup** para procurar na página pelo link do arquivo que deseja baixar. Utilize o comando abaixo para criar o seu objeto *soup* e procurar por todas as tags 'a'no documento:
 
 
```python
soup = BeautifulSoup(html, "html.parser")
links = soup.find_all('a')
```

Uma dica para baixar apenas os arquivos do mês de junho é verificar a data no nome do arquivo. Por exemplo, para baixar o arquivo do dia 17/06/2017 verifique se o link termina com *"turnstile_170610.txt"*. Se não fizer isso você baixará todos os arquivos da página. Para fazer isso utilize o comando conforme abaixo:

```python
if '1706' in link.get('href'):
```

E a dica final é utilizar o comando abaixo para fazer o download do arquivo txt:

```python
urllib.urlretrieve(link_do_arquivo, filename)
```

Lembre-se, primeiro, carregue todos os pacotes e funções que você estará usando em sua análise.

In [1]:
# N O T A : =======================================================================================================
# Notei que, por diversas vezes, a página da MTA está offline (provavelmente atualizando arquivos de controle do dia)
# Dessa forma, criei uma função para baixar os arquivos a partir de um link permanente
# disponibilizado na página do projeto na Udacity

from urllib.request import urlopen, urlretrieve
import os
import time

def RedirecDownloadEspelho():
# Redirecionamento se o servidor da MTA estiver offline

    dir_atual = os.getcwd()

    print("Servidor principal da MTA offline. Utilizando link alternativo...")
    print("-----------------------------------------------------------------")

    mirror = 'https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/KEY_turnstile-1706DAY/turnstile-1706DAY.txt'

    keys = ['5bf31d11', '5bf2fab0', '5bf31e03','5bf32142']
    days = ['03', '10', '17', '24']

    ext = mirror.split('.')[-1]

    # Cronômetro
    tempo_ini = time.time()

    for i in range(len(days)):
        # Atualiza iterativamente os links e nomes dos arquivos
        tmp_mirror = mirror.replace('KEY', keys[i])
        tmp_mirror = tmp_mirror.replace('DAY', days[i])
        filename = "dados_" + tmp_mirror.split("/")[-1].replace('-','_')
        urlretrieve(tmp_mirror, filename)
        print("Baixado arquivo do link {} \nSalvo em {}\{} ".format(tmp_mirror, dir_atual, filename))

    tempo_fim = time.time()
    tempo_dur = (tempo_fim - tempo_ini)
    m, s = divmod(tempo_dur, 60)

    print("Tempo para recuperação dos arquivos via URL: {:.0f}min {:.0f}s".format(m, s))

In [2]:
# Bibliotecas necessárias:
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup

import os
import time
import sys

dir_atual = os.getcwd()
string_interesse = '1706'

url = "http://web.mta.info/developers/turnstile.html"

u = urlopen(url)
html = u.read()
soup = BeautifulSoup(html, "html.parser")

# Lista os links referenciando arquivos na estrutura html da página da MTA
lista_link = soup.find_all('a')

tempo_ini = time.time()

flagErro = 0
for link in lista_link:
    
    if flagErro == 0:    
        if string_interesse in str(link.get('href')):

            filename = "dados_" + str(link.get('href')).split("/")[-1]
            # Compõe os links com base na url e nas referências a arquivos encontradas
            link_url = '/'.join(str(u.geturl()).split('/')[:4]) + '/' + str(link.get('href'))      

            # Como a página da MTA recorrentemente está offline, inseri este controle de erros com
            # 'try' e 'except'
            try:
                urlretrieve(link_url, filename)
                raise NotImplementedError("Conexão conseguida!")            
                print("Baixado arquivo do link {} \nSalvo em {}\{} ".format(link_url, dir_atual, filename))

                tempo_fim = time.time()
                tempo_dur = (tempo_fim - tempo_ini)
                m, s = divmod(tempo_dur, 60)
                print("Tempo para recuperação dos arquivos via URL: {:.0f}min {:.0f}s".format(m, s))

            except Exception as e:
                flagErro = 1
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print("Aconteceu um erro tipo {} registrado em {}, a partir da linha {}".format(exc_type, 
                                                                                                fname, 
                                                                                                exc_tb.tb_lineno-1))
                RedirecDownloadEspelho()                
                os._exit
    else:        
        os._exit

Aconteceu um erro tipo <class 'NotImplementedError'> registrado em <ipython-input-2-65b61a876e88>, a partir da linha 36
Servidor principal da MTA offline. Utilizando link alternativo...
-----------------------------------------------------------------
Baixado arquivo do link https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf31d11_turnstile-170603/turnstile-170603.txt 
Salvo em C:\Users\Usuario\Documents\Python Scripts\Nanodegree - Data Scientist\Proj3 - BigData - Metrô de NY\dados_turnstile_170603.txt 
Baixado arquivo do link https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf2fab0_turnstile-170610/turnstile-170610.txt 
Salvo em C:\Users\Usuario\Documents\Python Scripts\Nanodegree - Data Scientist\Proj3 - BigData - Metrô de NY\dados_turnstile_170610.txt 
Baixado arquivo do link https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf31e03_turnstile-170617/turnstile-170617.txt 
Salvo em C:\Users\Usuario\Documents\Python Scripts\

### *Exercicio 1.2*

Escreva uma função que pegue a lista de nomes dos arquivos que você baixou no exercicio 1.1 e consolide-os em um único arquivo. Deve existir apenas uma linha de cabeçalho no arquivo de saida. 

Por exemplo, se o arquivo_1 tiver:
linha 1...
linha 2...

e o outro arquivo, arquivo_2 tiver:
linha 3...
linha 4...
linha 5...

Devemos combinar o arquivo_1 com arquivo_2 em um arquivo mestre conforme abaixo:

'C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn'
linha 1...
linha 2...
linha 3...
linha 4...
linha 5...

**OBS:** Note que algumas colunas foram descartadas!

In [3]:
def create_master_turnstile_file(filenames, output_file):
    with open(output_file, 'w') as master_file:
        master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
        for filename in filenames:
            with open(filename) as arq_parcial:
                delim = ','
                for linha in arq_parcial:
                    # Pular cabeçalho repetido e dados de outros meses
                    if linha.startswith("C/A"): #or linha.split(delim)[6]:
                        continue
                    # Descarta as colunas 'STATION','LINENAME' e 'DIVISION', que não são de interesse :
                    master_file.write(delim.join(linha.split(delim)[0:3]) + delim + delim.join(linha.split(delim)[6:11]) + '\n')                      
    master_file.close()

In [4]:
# Lista os arquivos baixados
string_interesse = '1706'
ext = 'txt'
lista_arq = [arq for arq in os.listdir() if arq.startswith('dados_') and string_interesse in arq and arq.endswith(ext)]
    
create_master_turnstile_file(lista_arq, "dados_MTA_jun2017.txt")

### *Exercicio 1.3*

Neste exercício, escreva um função que leia o master_file criado no exercicio anterior e carregue-o em um pandas dataframe. Esta função deve filtrar para que o dataframe possua apenas linhas onde a coluna "DESCn" possua o valor "Regular".

Por exemplo, se o data frame do pandas estiver conforme abaixo:
    
    ,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn
    0,A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,3144312,1088151
    1,A002,R051,02-00-00,05-01-11,04:00:00,DOOR,3144335,1088159
    2,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177
    3,A002,R051,02-00-00,05-01-11,12:00:00,DOOR,3144424,1088231

O dataframe deverá ficar conforme abaixo depois de filtrar apenas as linhas onde a coluna DESCn possua o valor REGULAR:

    0,A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,3144312,1088151
    2,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177


In [5]:
import pandas as pd

def filter_by_regular(filename):    
    turnstile_data = pd.read_csv(filename, dtype= object) 
    turnstile_data = turnstile_data[turnstile_data['DESCn']== 'REGULAR']
    return turnstile_data

In [6]:
df = filter_by_regular("dados_MTA_jun2017.txt")
df.head(n=5)

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn
0,A002,R051,02-00-00,05/27/2017,00:00:00,REGULAR,0006195217,0002098317 ...
1,A002,R051,02-00-00,05/27/2017,04:00:00,REGULAR,0006195240,0002098318 ...
2,A002,R051,02-00-00,05/27/2017,08:00:00,REGULAR,0006195256,0002098347 ...
3,A002,R051,02-00-00,05/27/2017,12:00:00,REGULAR,0006195346,0002098432 ...
4,A002,R051,02-00-00,05/27/2017,16:00:00,REGULAR,0006195518,0002098491 ...


### *Exercicio 1.4*


Os dados do metrô de NY possui dados cumulativos de entradas e saidas por linha. Assuma que você possui um dataframe chamado df que contém apenas linhas para uma catraca em particular (unico SCP, C/A, e UNIT). A função abaixo deve alterar essas entradas cumulativas para a contagem de entradas desde a última leitura (entradas desde a última linha do dataframe).

Mais especificamente, você deverá fazer duas coisas:

1. Criar uma nova coluna chamada ENTRIESn_hourly

2. Inserir nessa coluna a diferença entre ENTRIESn da linha atual e a da linha anterior. Se a linha possuir alguma NAN, preencha/substitua por 1.

Dica: as funções do pandas shift() e fillna() pode ser úteis nesse exercicio.

Abaixo tem um exemplo de como seu dataframe deve ficar ao final desse exercicio:

           C/A  UNIT       SCP     DATEn     TIMEn    DESCn  ENTRIESn    EXITSn  ENTRIESn_hourly
    0     A002  R051  02-00-00  05-01-11  00:00:00  REGULAR   3144312   1088151                1
    1     A002  R051  02-00-00  05-01-11  04:00:00  REGULAR   3144335   1088159               23
    2     A002  R051  02-00-00  05-01-11  08:00:00  REGULAR   3144353   1088177               18
    3     A002  R051  02-00-00  05-01-11  12:00:00  REGULAR   3144424   1088231               71
    4     A002  R051  02-00-00  05-01-11  16:00:00  REGULAR   3144594   1088275              170
    5     A002  R051  02-00-00  05-01-11  20:00:00  REGULAR   3144808   1088317              214
    6     A002  R051  02-00-00  05-02-11  00:00:00  REGULAR   3144895   1088328               87
    7     A002  R051  02-00-00  05-02-11  04:00:00  REGULAR   3144905   1088331               10
    8     A002  R051  02-00-00  05-02-11  08:00:00  REGULAR   3144941   1088420               36
    9     A002  R051  02-00-00  05-02-11  12:00:00  REGULAR   3145094   1088753              153
    10    A002  R051  02-00-00  05-02-11  16:00:00  REGULAR   3145337   1088823              243

In [7]:
import pandas as pd
import numpy as np

delim = "_"

def get_hourly_entries(df):
    
    Rotulo = 'ENTRIESn'
    NaN_Substit = 1
    
    # Define uma chave concatenando 'C/A', 'UNIT' e 'SCP' 
    # para verificar variações de entradas na mesma chave: 
    UniqueKey = df['C/A'] + delim + df['UNIT'] + delim + df['SCP']
                                      # Se o grupo é o mesmo,
    df['ENTRIESn_hourly'] = np.where((UniqueKey == UniqueKey.shift(1)),
                                     # calcula a diferença entre volume de entradas
                                     pd.to_numeric(df[Rotulo]) - pd.to_numeric(df[Rotulo].shift(1)),
                                     # se não, marca como 'NaN'
                                     float('NaN'))
    # Substitui 'NaN'
    df.fillna(NaN_Substit, inplace= True)
    # Exibe números no formato inteiro
    pd.options.display.float_format = '{:,.0f}'.format
    return df

In [8]:
get_hourly_entries(df).head(n=5)

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly
0,A002,R051,02-00-00,05/27/2017,00:00:00,REGULAR,0006195217,0002098317 ...,1
1,A002,R051,02-00-00,05/27/2017,04:00:00,REGULAR,0006195240,0002098318 ...,23
2,A002,R051,02-00-00,05/27/2017,08:00:00,REGULAR,0006195256,0002098347 ...,16
3,A002,R051,02-00-00,05/27/2017,12:00:00,REGULAR,0006195346,0002098432 ...,90
4,A002,R051,02-00-00,05/27/2017,16:00:00,REGULAR,0006195518,0002098491 ...,172


### *Exercicio 1.5*

Faça o mesmo do exercicio anterior mas agora considerando as saidas, coluna EXITSn.
Para isso crie uma coluna chamada de EXITSn_hourly e insira a diferença entre a coluna EXITSn da linha atual versus a linha anterior. Se tiver algum NaN, preencha/substitua por 0.



In [9]:
import pandas as pd
import numpy as np

def get_hourly_exits(df):  
    
    Rotulo = 'EXITSn'
    NaN_Substit = 0
    
    # Define uma chave concatenando 'C/A', 'UNIT' e 'SCP' 
    # para verificar variações de entradas na mesma chave:
    UniqueKey = df['C/A'] + delim + df['UNIT'] + delim + df['SCP']    
                                      # Se o grupo é o mesmo,
    df['EXITSn_hourly'] = np.where((UniqueKey == UniqueKey.shift(1)),
                                     # calcula a diferença entre volume de saídas
                                     pd.to_numeric(df[Rotulo]) - pd.to_numeric(df[Rotulo].shift(1)),
                                     # se não, marca como 'NaN'
                                     float('NaN'))
    # Substitui 'NaN'
    df.fillna(NaN_Substit, inplace= True)
    # Exibe números no formato inteiro
    pd.options.display.float_format = '{:,.0f}'.format
    return df

In [10]:
get_hourly_exits(df).head(n=5)

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly
0,A002,R051,02-00-00,05/27/2017,00:00:00,REGULAR,0006195217,0002098317 ...,1,0
1,A002,R051,02-00-00,05/27/2017,04:00:00,REGULAR,0006195240,0002098318 ...,23,1
2,A002,R051,02-00-00,05/27/2017,08:00:00,REGULAR,0006195256,0002098347 ...,16,29
3,A002,R051,02-00-00,05/27/2017,12:00:00,REGULAR,0006195346,0002098432 ...,90,85
4,A002,R051,02-00-00,05/27/2017,16:00:00,REGULAR,0006195518,0002098491 ...,172,59


### *Exercicio 1.6*

Dado uma variável de entrada que representa o tempo no formato de:
     "00:00:00" (hora: minutos: segundos)
    
Escreva uma função para extrair a parte da hora do tempo variável de entrada e devolva-o como um número inteiro. Por exemplo:
         
         1) se a hora for 00, seu código deve retornar 0
         2) se a hora for 01, seu código deve retornar 1
         3) se a hora for 21, seu código deve retornar 21
        
Por favor, devolva a hora como um número inteiro.


In [11]:
def time_to_hour(time):
    # Seleciona a primeira porção de string antes de ':' e converte para inteiro
    hour = time.map(lambda t: int(t.split(":")[0]))
    return hour

time_to_hour(df['TIMEn']).head(n=5)

0     0
1     4
2     8
3    12
4    16
Name: TIMEn, dtype: int64

## Exercicio 2 - Análise dos dados

### *Exercicio 2.1*

Para verificar a relação entre o movimento do metrô e o clima, precisaremos complementar os dados do arquivo já baixado com os dados do clima.
Nós complementamos para você este arquivo com os dados de clima de Nova Iorque  e disponibilizamos na área de materiais do projeto. Você pode acessa-lo pelo link: https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv

Agora que temos nossos dados em um arquivo csv, escreva um código python que leia este arquivo e salve-o em um data frame do pandas. 

Dica: 

Utilize o comando abaixo para ler o arquivo:

```python
pd.read_csv('output_list.txt', sep=",")
```



In [12]:
import pandas as pd
import os
import time

dir_atual = os.getcwd()
filename = "turnstile_data_master_with_weather.csv"
link_parcial = "https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/"
link_completo = link_parcial + filename

ext = link_completo.split('.')[-1]

# Cronômetro
tempo_ini = time.time()
   
urlretrieve(link_completo, filename)
print("Baixado arquivo do link {} \nSalvo em {}\{} ".format(link_completo, dir_atual, filename))

tempo_fim = time.time()
tempo_dur = (tempo_fim - tempo_ini)

print("Tempo para recuperação do arquivo via URL: {:.0f}s".format(tempo_dur))

DadosClima = pd.read_csv(filename, sep= ",")
DadosClima.head(n=5)

Baixado arquivo do link https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv 
Salvo em C:\Users\Usuario\Documents\Python Scripts\Nanodegree - Data Scientist\Proj3 - BigData - Metrô de NY\turnstile_data_master_with_weather.csv 
Tempo para recuperação do arquivo via URL: 47s


,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,0,R001,2011-05-01,01:00:00,1,REGULAR,0,0,30,42,...,39,30,0,0,5,50,60,69,0,0
1,1,R001,2011-05-01,05:00:00,5,REGULAR,217,553,30,42,...,39,30,0,0,5,50,60,69,0,0
2,2,R001,2011-05-01,09:00:00,9,REGULAR,890,"1,262",30,42,...,39,30,0,0,5,50,60,69,0,0
3,3,R001,2011-05-01,13:00:00,13,REGULAR,"2,451","3,708",30,42,...,39,30,0,0,5,50,60,69,0,0
4,4,R001,2011-05-01,17:00:00,17,REGULAR,"4,400","2,501",30,42,...,39,30,0,0,5,50,60,69,0,0


### *Exercicio 2.2*

Agora crie uma função que calcule a quantidade de dias chuvosos, para isso retorne a contagem do numero de dias onde a coluna *"rain"* é igual a 1.

Dica: Você também pode achar que a interpretação de números como números inteiros ou float pode não
     funcionar inicialmente. Para contornar esta questão, pode ser útil converter
     esses números para números inteiros. Isso pode ser feito escrevendo cast (coluna como inteiro).
     Então, por exemplo, se queríamos lançar a coluna maxtempi como um número inteiro, nós devemos
     escrever algo como cast (maxtempi as integer) = 76, em oposição a simplesmente
     onde maxtempi = 76.

In [13]:
def num_rainy_days(df):
    df['rain'] = pd.to_numeric(df['rain'])
    # Faz-se o agrupamento por data para se contabilizar o dia uma única vez
    QtdeDiasChuvosos = len(df[df['rain'] == 1].groupby('DATEn'))
    return QtdeDiasChuvosos

num_rainy_days(DadosClima)

10

### *Exercicio 2.3*

Calcule se estava nebuloso ou não (0 ou 1) e a temperatura máxima para fog (isto é, a temperatura máxima 
     para dias nebulosos).

In [14]:
def max_temp_aggregate_by_fog(df):    
        
    MaxTempNeb = df[['fog', 'maxtempi']].groupby(['fog']).max()
    return MaxTempNeb
       
max_temp_aggregate_by_fog(DadosClima)

,maxtempi
fog,
0,86
1,81


### *Exercicio 2.4*

Calcule agora a média de 'meantempi' nos dias que são sábado ou domingo (finais de semana):

In [15]:
def avg_weekend_temperature(filename):
    
    df = pd.read_csv(filename, sep = ',', dtype= object) 
    # Converte uma data em dia da semana com Monday = 0 e Sunday = 6
    df['DiaSemana'] = pd.to_datetime(df['DATEn']).dt.dayofweek
    mean_temp_weekends = df[df['DiaSemana'] >= 5]['meantempi'].astype(float).mean()
    return mean_temp_weekends

avg_weekend_temperature('turnstile_data_master_with_weather.csv')

65.10066685403307

### *Exercicio 2.5*

Calcule a média da temperatura mínima 'mintempi' nos dias chuvosos onde da temperatura mínima foi maior que do 55 graus:

In [16]:
def avg_min_temperature(filename):
    
    df = pd.read_csv(filename, sep = ',', dtype= object) 
    avg_min_temp_rainy  = df[(df['rain'] == '1.0') & (df['mintempi'].astype(float) > 55.0)]['mintempi'].astype(float).mean() 
    return avg_min_temp_rainy

avg_min_temperature('turnstile_data_master_with_weather.csv')

61.238860398860396

### *Exercicio 2.6*

Antes de realizar qualquer análise, pode ser útil olhar para os dados que esperamos analisar. Mais especificamente, vamos examinar as entradas por hora em nossos dados do metrô de Nova York para determinar a distribuição dos dados. Estes dados são armazenados na coluna ['ENTRIESn_hourly'].
    
Trace dois histogramas nos mesmos eixos para mostrar as entradas quando esta chovendo vs quando não está chovendo. 
Abaixo está um exemplo sobre como traçar histogramas com pandas e matplotlib:
     
```python
Turnstile_weather ['column_to_graph']. Hist ()
```   
    

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def entries_histogram(turnstile_weather):   
        
    plt.figure()    
    for flag_rain in [0,1]:
        # Converte valores de entradas horárias e marcador de chuva em float
        turnstile_weather['ENTRIESn_hourly'] = pd.to_numeric(turnstile_weather['ENTRIESn_hourly'])
        turnstile_weather['rain'] = pd.to_numeric(turnstile_weather['rain'])

        # Seleciona o conjunto de dados segundo a condição de dia chuvoso ou não
        dados_hist = turnstile_weather[turnstile_weather['rain'] == flag_rain]['ENTRIESn_hourly']  

        # Aplica a Regra de Sturges para definir a quantidade adequada de intervalos de dados
        nbins = int((1 + 3.32*(np.log10(len(dados_hist)))))
        # Plota o histograma com opacidade nas colunas
        dados_hist.hist(alpha= .5, bins= nbins)

        plt.xlabel("Entradas por hora")
        plt.ylabel("Frequência")    
        plt.title('Frequência das entradas por hora em dias chuvosos e não chuvosos\n')  
        plt.legend(["Dias não chuvosos", "Dias chuvosos"])        
                
    plt.show()
    return plt

filename = "turnstile_data_master_with_weather.csv"
DadosClima = pd.DataFrame(pd.read_csv(filename, dtype=object))

entries_histogram(DadosClima)

<Figure size 640x480 with 1 Axes>

<module 'matplotlib.pyplot' from 'C:\\Users\\Usuario\\Anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>

### *Exercicio 2.7*

Os dados que acabou de plotar são de que tipo de distribuição? Existe diferença na distribuição entre dias chuvosos e não chuvosos?

** Resposta **:
Os dados em ambos cenários apresentam-se em uma **distribuição assimétrica positiva**, anormal. 
A curvatura de ambos os dados tem formato semelhante (pode-se testar sua aderência à distribuição exponencial, por exemplo), sendo a moda a diferença mais evidente entre os histogramas (intervalo de quantidade de entradas mais frequente); elas têm valores aproximadamente de 77.000 e 40.000 para dias chuvosos e não chuvosos respectivamente. Estes valores podem variar de acordo com o agrupamento no eixo horizontal (dado pelo parâmetro _bins_)

### *Exercicio 2.8*

Construa uma função que que retorne:

1. A média das entradas com chuva
2. A média das entradas sem chuva




In [18]:
import numpy as np

import pandas

def means(turnstile_weather):
    
    with_rain_mean    = turnstile_weather[turnstile_weather['rain'] == 1.0]['ENTRIESn_hourly'].mean()
    without_rain_mean = turnstile_weather[turnstile_weather['rain'] == 0  ]['ENTRIESn_hourly'].mean()
    
    # Deixei p como vazio para manter a saída desejada da função
    p = {}
    
    return with_rain_mean, without_rain_mean, p # leave this line for the grader

means(DadosClima)

(1105.4463767458733, 1090.278780151855, {})

Responda as perguntas abaixo de acordo com a saida das suas funções:

1. Qual a média das entradas com chuva?
2. Qual a média das entradas sem chuva?


** Resposta **:
1. A média das entradas com chuva foi de 1105.45.
2. A média das entradas sem chuva foi de 1090.28.

## Exercicio 3 - Map Reduce

### *Exercicio 3.1*

A entrada para esse exercício e o mesmo arquivo da seção anterior (Exercicio 2). Você pode baixar o arquivo neste link:

 https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv

Varmos criar um mapeador agora. Para cada linha de entrada, a saída do mapeador deve imprimir (não retornar) a UNIT como uma chave e o número de ENTRIESn_hourly como o valor. Separe a chave e o valor por uma guia. Por exemplo: 'R002 \ t105105.0'

Exporte seu mapeador em um arquivo chamado mapper_result.txt e envie esse arquivo juntamente com a sua submissão. O código para exportar seu mapeador já está escrito no código abaixo.




In [19]:
import sys

def mapper():
    
    delim = ','
    cabecalho = ",UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei, " \
                "meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder"
    
    # Encontra a posição de 'UNIT' e 'ENTRIESn_hourly' no cabeçalho e, portanto, nas colunas do arquivo      
    posUnit = [i for i, j in enumerate(cabecalho.split(delim)) if j == 'UNIT']
    posENTRIES = [i for i, j in enumerate(cabecalho.split(delim)) if j == 'ENTRIESn_hourly']
    
    
    sys.stdin = open('turnstile_data_master_with_weather.csv')
    for linha in sys.stdin:
        # Cria uma lista a partir do cabeçalho, separando por ','
        dados = linha.split(delim)
        
        # O valor de 'UNIT' está na posição 'posUNIT[0]'
        unit = dados[posUnit[0]]
        # e o valor de 'ENTRIESn_hourly' está na posição 'posENTRIES[0]'
        entries = dados[posENTRIES[0]]
        print("{}\t{}".format(unit, entries))        
        
sys.stdout = open('mapper_result.txt', 'w')
mapper()

### *Exercicio 3.2*

Agora crie o redutor. Dado o resultado do mapeador do exercicio anterior, o redutor deve imprimir (não retornar) uma linha por UNIT, juntamente com o número total de ENTRIESn_hourly ao longo de maio (que é a duração dos nossos dados), separados por uma guia. Um exemplo de linha de saída do redutor pode ser assim: 'R001 \ t500625.0'

Você pode assumir que a entrada para o redutor está ordenada de tal forma que todas as linhas correspondentes a uma unidade particular são agrupados. No entanto, a saída do redutor terá repetição pois existem lojas que aparecem em locais diferentes dos arquivos.

Exporte seu redutor em um arquivo chamado reducer_result.txt e envie esse arquivo juntamente com a sua submissão.

In [20]:
import sys

def reducer():    
    
    # Devido à forma como o mapeador foi gerado, não tendo se organizado as impressões de dados em ordem alfabética de UNIT,
    # o redutor aqui construído apresenta oportunidades de melhoria, já que as linhas por UNIT não serão únicas
    
    print("UNIT\tENTRIESn_hourly")
    EntriesTotal = 0
    KeyAnterior = None
    
    sys.stdin = open('mapper_result.txt')    
    linhas = sys.stdin.readlines()
    
    for l in range(1, len(linhas)):
        KeyAtual, EntriesAtual = linhas[l].split("\t")        
        # Se os identificadores entre as linhas atual e anteriores diferirem,
        if KeyAnterior and KeyAtual != KeyAnterior:
            # imprime os valores de chave e acumulado de entradas
            print("{}\t{}".format(KeyAnterior, EntriesTotal))           
            KeyAnterior = KeyAtual
            EntriesTotal = 0           

        KeyAnterior = KeyAtual
        EntriesAtual_float = float(EntriesAtual)            
        # acumula a contagem
        EntriesTotal += EntriesAtual_float
    
    # imprime a última linha
    if KeyAnterior != None:
        print("{}\t{}".format(KeyAnterior, EntriesTotal))
            
sys.stdout = open('reducer_result.txt', 'w')
reducer()